In [1]:
import os
import numpy as np
import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns
# import pingouin as pg
from scipy import stats

In [2]:
def plot_corr(df, X, Y, out_path, fig_title=None, fig_size=(5, 5), 
              x_name=None, y_name=None, x_lim=None, y_lim=None):
    N = len(df)
    plt.figure(figsize=fig_size, dpi=500)
    sns.set(style="whitegrid", font_scale=1.2)

    ax = sns.JointGrid(data=df, x=df[X].to_numpy(), y=df[Y].to_numpy(),
                       xlim=x_lim, ylim=y_lim, height=5, ratio=3)
    ax = ax.plot_joint(sns.regplot, x_jitter=False, y_jitter=False, 
                       scatter_kws={"alpha": 0.5, "edgecolor": "white"}, 
                       line_kws={"linewidth": 1})
    ax = ax.plot_marginals(sns.histplot, kde=True, binwidth=2, linewidth=1)
    
    ax.refline(x=21.33, color="r", alpha=.7)
    ax.ax_joint.text(x=23, y=max(df["Y"])+1, 
                     s="threshold", color="r", fontstyle="italic", size=10)
    ax.refline(y=0, color="r", alpha=.7)
    ax.refline(y=np.mean(df[Y]), color="g")
    ax.ax_joint.text(x=3, y=np.mean(df["Y"])+1, 
                     s="mean", color="g", fontstyle="italic", size=10)
    
    r, p = stats.pearsonr(df[X], df[Y], alternative="two-sided")
    p_txt = "p < .001" if p < .001 else "p = "+"{:.3f}".format(p).lstrip("0")
    p_color = "r" if p < .05 else "k"
    p_fw = "bold" if p < .05 else "normal"
    ax.ax_joint.text(x=3, y=min(df["Y"]), 
                     s="r = {:.2f}, {:}".format(r, p_txt), 
                     fontstyle="italic", color=p_color, fontweight=p_fw)
    
    ax.ax_joint.text(x=45, y=max(df["Y"]), 
                     s=f"N = {N}", color="gray", fontstyle="italic", size=12)
    ax.set_axis_labels(xlabel=x_name, 
                       ylabel=y_name, 
                       size=12)
    if fig_title != None:
        # plt.title(fig_title, fontsize=20)
        plt.suptitle(fig_title, fontsize=14, x=.85, y=.85)
    plt.tight_layout()
    plt.savefig(out_path, format="png", dpi=500)
    plt.close()
    plt.clf()
    
    return r, p

In [3]:
def plot_heatmap(mat, x_labs, y_labs, out_path, fig_title=None, fig_size=(10, 20), 
                 cmap=["coolwarm", "Spectral_r"][0], cmin=-1, cmax=1, 
                 norm=None, show_value=True, fmt=".3f"):
        
    plt.figure(figsize=fig_size, dpi=500)
    sns.set(style="white", font_scale=1.2)
    ax = sns.heatmap(mat, cmap=cmap, # square=True, 
                     annot=show_value, fmt=fmt, 
                     vmin=cmin, vmax=cmax, 
                     linewidth=.5, norm=norm, 
                     cbar_kws={"shrink": 0.5})
#     ax.xaxis.set_major_locator(mpl.ticker.FixedLocator(np.arange(0, len(x_labs))))
#     ax.yaxis.set_major_locator(mpl.ticker.FixedLocator(np.arange(0, len(y_labs))))
    ax.set_xticklabels(x_labs, rotation=90, fontsize=20)
    ax.set_yticklabels(y_labs, rotation=0, fontsize=20)
#     ax.set_title(fig_title, fontsize=20)
    plt.suptitle(fig_title, fontsize=20, 
                 x=.95, y=.95, horizontalalignment="right")
    plt.tight_layout()
    plt.savefig(out_path, format="png")
    plt.close()
    plt.clf()

In [4]:
def remove_outlier_indices(df, col):
    Q1 = df[col].quantile(0.25)
    Q3 = df[col].quantile(0.75)
    IQR = Q3 - Q1
    not_outliers = ~((df[col] < (Q1 - 1.5 * IQR)) | (df[col] > (Q3 + 1.5 * IQR)))
    return not_outliers

In [5]:
re_plot = True

ap_ver = "3ord_v3"
reml = ["", "_REML"][1]
cond_list = ["Fst", "Snd", "Trd"]
run_list = range(2, 10)

In [6]:
# roi_list = ["Sub", "CA1", "CA2_CA3", "DG_CA4"]
roi_list = [
    "PreCent", "ParaCent", 
    "PostCent", "SupMarg", "SupPari", "InfPari", "PreCune", 
    "Lingual", "PeriCal", "Cuneus", 
    "V1", "V2", "V3", "high_Vis", 
    "SupTemp", "MidTemp", "InfTemp", "TranTemp", "Fusiform", 
    "PHC", "PRC", "ERC", 
    "Sub", "CA1", "CA2_CA3", "DG_CA4", 
    "Caud", "Puta", 
    "Insula", "PostCing", "IsthCing"
]

In [7]:
# top_dir = os.path.join("/media", "data2", "pinwei", "SL_hippocampus")
# data_dir = os.path.join(top_dir, "Combined_group",  "derivatives", "RBA", f"runs_MNI_{ap_ver}", f"dataSet{reml}")
# out_top = os.path.join(top_dir, "Combined_group",  "derivatives", "ROI_behav_corr", f"{ap_ver}{reml}")

In [8]:
data_dir = os.path.join("RBA", "runs_MNI_3ord_v3", "dataSet_REML")
out_top = "ROI_behav_corr"

In [ ]:
for cond in cond_list:
    fig_folder = os.path.join(out_top, cond)
    if not os.path.exists(fig_folder):
        os.makedirs(fig_folder)

    data_name = f"data_{cond}.txt"
    data_table = pd.read_csv(os.path.join(data_dir, data_name), sep="\t", header=0)
        # col_names = ["Subj", "Age", "Design", "Score", "Guess", "ROI", "Y", "tval", "Run"]
    data_table["Score"] = data_table["Score"] *60
    data_table["Y"] = data_table["Y"] *100

    corr_R, corr_pval = {}, {}

    for run in run_list:
        # fig_folder = os.path.join(out_top, cond, "run-{:02d}".format(run))
        # if not os.path.exists(fig_folder):
        #     os.makedirs(fig_folder)

        corr_R[run], corr_pval[run] = {}, {} 

        for roi in roi_list:
            # sub_df = data_table.query("ROI == @roi & Run == @run")
            sub_df = data_table[(data_table["ROI"] == roi) & (data_table["Run"] == run)]
            sub_df = sub_df[["Y", "Score"]]
            sub_df = sub_df[remove_outlier_indices(sub_df, "Y")]
            fig_name = "{:}_run-{:02d}.png".format(roi, run)

            r, p = plot_corr(sub_df, "Score", "Y", 
                                 fig_title="{:}\nrun-{:02d}".format(roi, run), 
                                 out_path=os.path.join(fig_folder, fig_name), 
                                 x_name="Familiarity score", 
                                 y_name=f"Intensity (×100)", 
                                 x_lim=(0, 60))
            
            corr_R[run][roi], corr_pval[run][roi] = r, p

    for corr_dict, typ in zip([corr_R, corr_pval], ["R", "pVal"]):
        mat = pd.DataFrame(corr_dict)
        file_name = "{:}_cormat_{:}.csv".format(typ, cond)
        file_path = os.path.join(out_top, file_name)
        mat.to_csv(file_path)

#         fig_path = file_path.replace(".csv", ".png")
#         if not os.path.isfile(fig_path) or re_plot:
#             plot_heatmap(mat, 
#                          x_labs=[ "run-{:02}".format(run) for run in run_list ], 
#                          y_labs=roi_list,
#                          cmap=["coolwarm", "hot"][xi], 
#                          cmin=[-1, 0][xi], cmax=[1, 1][xi],
#                          out_path=fig_path, 
#                          fig_title="{:}values, {:}".format(["R ", "p-"], cond))

C:\Users\PinWei\anaconda3\lib\site-packages\seaborn\axisgrid.py:1690: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  f = plt.figure(figsize=(height, height))
<ipython-input-2-050ec86a9c69>:4: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  plt.figure(figsize=fig_size, dpi=500)
C:\Users\PinWei\anaconda3\lib\site-packages\seaborn\axisgrid.py:1690: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To co

<ipython-input-2-050ec86a9c69>:4: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  plt.figure(figsize=fig_size, dpi=500)
C:\Users\PinWei\anaconda3\lib\site-packages\seaborn\axisgrid.py:1690: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  f = plt.figure(figsize=(height, height))
<ipython-input-2-050ec86a9c69>:4: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam 

<ipython-input-2-050ec86a9c69>:4: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  plt.figure(figsize=fig_size, dpi=500)
C:\Users\PinWei\anaconda3\lib\site-packages\seaborn\axisgrid.py:1690: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  f = plt.figure(figsize=(height, height))
<ipython-input-2-050ec86a9c69>:4: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam 

<ipython-input-2-050ec86a9c69>:4: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  plt.figure(figsize=fig_size, dpi=500)
C:\Users\PinWei\anaconda3\lib\site-packages\seaborn\axisgrid.py:1690: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  f = plt.figure(figsize=(height, height))
<ipython-input-2-050ec86a9c69>:4: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam 

<ipython-input-2-050ec86a9c69>:4: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  plt.figure(figsize=fig_size, dpi=500)
C:\Users\PinWei\anaconda3\lib\site-packages\seaborn\axisgrid.py:1690: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  f = plt.figure(figsize=(height, height))
<ipython-input-2-050ec86a9c69>:4: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam 

...

<ipython-input-2-050ec86a9c69>:4: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  plt.figure(figsize=fig_size, dpi=500)
C:\Users\PinWei\anaconda3\lib\site-packages\seaborn\axisgrid.py:1690: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  f = plt.figure(figsize=(height, height))
<ipython-input-2-050ec86a9c69>:4: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam 

<ipython-input-2-050ec86a9c69>:4: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  plt.figure(figsize=fig_size, dpi=500)
C:\Users\PinWei\anaconda3\lib\site-packages\seaborn\axisgrid.py:1690: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  f = plt.figure(figsize=(height, height))
<ipython-input-2-050ec86a9c69>:4: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam 

<ipython-input-2-050ec86a9c69>:4: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  plt.figure(figsize=fig_size, dpi=500)
C:\Users\PinWei\anaconda3\lib\site-packages\seaborn\axisgrid.py:1690: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  f = plt.figure(figsize=(height, height))
<ipython-input-2-050ec86a9c69>:4: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam 

<ipython-input-2-050ec86a9c69>:4: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  plt.figure(figsize=fig_size, dpi=500)
C:\Users\PinWei\anaconda3\lib\site-packages\seaborn\axisgrid.py:1690: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  f = plt.figure(figsize=(height, height))
<ipython-input-2-050ec86a9c69>:4: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam 

<ipython-input-2-050ec86a9c69>:4: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  plt.figure(figsize=fig_size, dpi=500)
C:\Users\PinWei\anaconda3\lib\site-packages\seaborn\axisgrid.py:1690: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  f = plt.figure(figsize=(height, height))
<ipython-input-2-050ec86a9c69>:4: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam 

In [ ]:
# for cond in cond_list:
#     for xi, typ in enumerate(["R", "pVal"]):
#         file_name = "{:}_cormat_{:}.csv".format(typ, cond)
#         file_path = os.path.join(out_top, file_name)
#         mat = pd.read_csv(file_path, header=0, index_col=0)

#         fig_path = file_path.replace(".csv", ".png")
#         plot_heatmap(mat, 
#                      x_labs=[ "run-{:02}".format(run) for run in run_list ], 
#                      y_labs=roi_list,
#                      cmap=["coolwarm", "hot"][xi], 
#                      cmin=[-1, 0][xi], cmax=[1, 1][xi],
#                      out_path=fig_path, 
#                      fig_title="{:}values\n{:}".format(["R ","p-"][xi], cond))

In [ ]:
# from matplotlib.colors import LogNorm

# for cond in cond_list:
#     typ = "pVal"
#     file_name = "{:}_cormat_{:}.csv".format(typ, cond)
#     file_path = os.path.join(out_top, file_name)
#     mat = pd.read_csv(file_path, header=0, index_col=0)

#     fig_path = file_path.replace(".csv", ".png")
#     plot_heatmap(mat, 
#                  x_labs=[ "run-{:02}".format(run) for run in run_list ], 
#                  y_labs=roi_list,
#                  cmap="hot", cmin=0, cmax=1, norm=LogNorm(), 
#                  out_path=fig_path, 
#                  fig_title="{:}values\n{:}".format("p-", cond))

In [ ]:
# sub_df.describe()